In [1]:
#!pip install tweepy
#!pip install --upgrade watson-developer-cloud

    100% |████████████████████████████████| 133kB 5.8MB/s eta 0:00:01
  Running setup.py bdist_wheel for oauthlib ... done
  Stored in directory: /gpfs/fs01/user/se1f-2003f1257f20bb-7f578b9e48a9/.cache/pip/wheels/e5/46/f7/bb2fde81726295a13a71e3c6396d362ab408921c6562d6efc0
Successfully built oauthlib
    100% |████████████████████████████████| 143kB 5.6MB/s eta 0:00:01
Requirement already up-to-date: requests<3.0,>=2.0 in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-developer-cloud)
    100% |████████████████████████████████| 51kB 6.8MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 7.2MB/s eta 0:00:01
Requirement already up-to-date: python_dateutil>=2.5.3 in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-developer-cloud)
Requirement already up-to-date: chardet<3.1.0,>=3.0.2 in /usr/local/src/conda3_runtime.v28/home/envs/DSX-Python35-Spark/lib/python3.5/sit

# Personality Insights

https://console.bluemix.net/docs/services/personality-insights/models.html#models

## Librerias

In [64]:
import re
import tweepy
import json
import pprint
import math
from tweepy import OAuthHandler
from watson_developer_cloud import PersonalityInsightsV3
import pandas as pd
from functools import reduce
#from textblob import TextBlob

## Extraccion de tweets

Usaremos los tweets para formar el corpus de prueba que enviaremos al API.

In [1]:
#coloque aqui Credenciales Twitter

In [3]:
# The code was removed by DSX for sharing.

In [4]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())  
    
    def parse_tweets(self, fetched_tweets):
        tweets = []
        
        for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['content'] = tweet.text
                #parsed_tweet['clean_text'] = self.clean_tweet(tweet.text)
                parsed_tweet['contenttype'] = 'text/plain' 
                parsed_tweet['language'] = tweet.lang
                parsed_tweet['id'] = tweet.id_str
                parsed_tweet['created'] = math.floor(tweet.created_at.timestamp())
                
                #parsed_tweet = tweet
                
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
                    
        return tweets
        
    def get_raw_tweets(self,query,count=10):        
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            tweets = self.parse_tweets(fetched_tweets)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))
            
    def get_user_tweets(self, userid, count):
        tweets=[]
        
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.user_timeline(id=userid,count=200)
 
            # parsing tweets one by one
            tweets = self.parse_tweets(fetched_tweets)
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

Teniendo los metodos, podemos extraer los tweets para un id de usuario particular

In [5]:
# creating object of TwitterClient ClassBogota
api = TwitterClient()
# calling function to get tweets
tweets = api.get_user_tweets('carlosvives',count=1000)

tweets[1:3] #mostramos los primeros 3

[{'content': 'RT @samuelazout: Los próximos emprendedores del cambio social en Santa Marta. #FutbolYDesarrollo #TodosPorPescaito #EducacionDeCalidad #Jov…',
  'contenttype': 'text/plain',
  'created': 1520422416,
  'id': '971257538405597185',
  'language': 'es'},
 {'content': 'los vivi completicos!!grandes!!!! https://t.co/QgFCAsP0MB',
  'contenttype': 'text/plain',
  'created': 1520373717,
  'id': '971053280083226625',
  'language': 'es'}]

Debemos poner los tweet en un formato especifico para enviarlos a watson

In [6]:
body = {"contentItems" : tweets}

## Calculamos el perfil

Ahora que tenemos los tweet procesados, podemos usar la API de Personality Insights. Primero creamos el objeto que permite enviar peticiones.

In [ ]:
#coloque aqui Credenciales NLU

In [ ]:
# The code was removed by DSX for sharing.

In [7]:
personality_insights = PersonalityInsightsV3(
    version='2016-10-20',
    username = nlu_username,
password = nlu_password)

La funcion `profile` devuelve el perfil en formato json

In [8]:
profile = personality_insights.profile(
    body, 
    content_type='application/json',
    accept_language = 'es')

In [9]:
pp = pprint.PrettyPrinter(indent=4)

In [10]:
pp.pprint(profile)

{   'behavior': [   {   'category': 'behavior',
                        'name': 'Domingo',
                        'percentage': 1.0,
                        'trait_id': 'behavior_sunday'},
                    {   'category': 'behavior',
                        'name': 'Lunes',
                        'percentage': 0.0,
                        'trait_id': 'behavior_monday'},
                    {   'category': 'behavior',
                        'name': 'Martes',
                        'percentage': 0.0,
                        'trait_id': 'behavior_tuesday'},
                    {   'category': 'behavior',
                        'name': 'Miércoles',
                        'percentage': 0.0,
                        'trait_id': 'behavior_wednesday'},
                    {   'category': 'behavior',
                        'name': 'Jueves',
                        'percentage': 0.0,
                        'trait_id': 'behavior_thursday'},
                    {   'category': 'behavior'

Encontramos
* Necesidades (needs)
* Personalidad (personality). Calculada como el Big5
* Valores
* Tendencias de consumo

## Como Interpretar los score?

## Que hacer con el perfil?

Graficas por supuesto

In [11]:
big5 = pd.DataFrame(list(map( lambda x: {'percentil':x['percentile'], 'nombre': x['name']}, profile['personality'])))
values = pd.DataFrame(list(map( lambda x: {'percentil':x['percentile'], 'nombre': x['name']}, profile['values'])))
needs = pd.DataFrame(list(map( lambda x: {'percentil':x['percentile'], 'nombre': x['name']}, profile['needs'])))

In [12]:
big5.sort_values('percentil', ascending = False)

,nombre,percentil
3,Amabilidad,0.993957
0,Apertura a experiencias,0.982688
1,Responsabilidad,0.981083
2,Extroversión,0.818419
4,Rango emocional,0.479767


In [13]:
values

,nombre,percentil
0,Conservación,0.975513
1,Apertura al cambio,0.488599
2,Hedonismo,0.220593
3,Superación personal,0.114810
4,Autotranscendencia,0.412704


In [14]:
needs

,nombre,percentil
0,Desafío,0.583199
1,Familiaridad,0.943182
2,Curiosidad,0.202411
3,Entusiasmo,0.407125
4,Armonía,0.893983
5,Ideal,0.099839
6,Libertad,0.737988
7,Amor,0.554905
8,Practicidad,0.643062
9,Autoexpresión,0.012693


Podemos usar estas features para aumentar modelos convencionales de machine learning

In [15]:
big5['user'] = 'carlosvives'
big5 = big5.pivot(index='user', columns='nombre', values='percentil')

In [16]:
big5

nombre,Amabilidad,Apertura a experiencias,Extroversión,Rango emocional,Responsabilidad
user,,,,,
carlosvives,0.993957,0.982688,0.818419,0.479767,0.981083


Escalemos esto a mas usuarios

In [73]:
def get_profile(user):
    print(user)
    tweets = api.get_user_tweets(user, count=1000)
    body = {"contentItems" : tweets}
    profile = personality_insights.profile(
        body, 
        content_type='application/json',
        accept_language = 'es')
    
    profile['user'] = user
    
    return profile

#aspect puede ser personality, needs, values
def aspect_table(profile, aspect):
    data = pd.DataFrame(list(map( lambda x: {'percentil':x['percentile'], 'nombre': x['name']}, profile[aspect])))
    data['user'] = profile['user']
    return data

def all_aspects_table(profile):
    big5 = aspect_table(profile, 'personality')
    values = aspect_table(profile, 'values')
    needs = aspect_table(profile, 'needs')
    
    all_aspects = big5.append(values, ignore_index=True)
    all_aspects = all_aspects.append(needs, ignore_index=True)
    
    return all_aspects

In [55]:
usuarios = ['jamesdrodriguez', 'FALCAO', 'juanes', 'SofiaVergara', 'JBALVIN', 'carlosvives', 'jessicacediel', 'LAAZCARATE', 'susoelpaspi', 'Cuadrado','lopezandres','ninagarcia','Soymarioruiz','D_Ospina1','CaroCruzO','juanjaramilloe','PLinero','SilvestreFDC','DanielSamperO','PirryTv','cejaspobladas','marianapajon']

In [56]:
perfiles = list(map(get_profile, usuarios))

jamesdrodriguez
FALCAO
juanes
SofiaVergara
JBALVIN
carlosvives
jessicacediel
LAAZCARATE
susoelpaspi
Cuadrado
lopezandres
ninagarcia
Soymarioruiz
D_Ospina1
CaroCruzO
juanjaramilloe
PLinero
SilvestreFDC
DanielSamperO
PirryTv
cejaspobladas
marianapajon


In [74]:
tablas_aspectos = list(map(all_aspects_table, perfiles))

In [75]:
tablas_aspectos[3]

,nombre,percentil,user
0,Apertura a experiencias,0.353916,SofiaVergara
1,Responsabilidad,0.896977,SofiaVergara
2,Extroversión,0.707688,SofiaVergara
3,Amabilidad,0.935896,SofiaVergara
4,Rango emocional,0.435184,SofiaVergara
5,Conservación,0.958378,SofiaVergara
6,Apertura al cambio,0.567176,SofiaVergara
7,Hedonismo,0.687010,SofiaVergara
8,Superación personal,0.885256,SofiaVergara
9,Autotranscendencia,0.708221,SofiaVergara


In [76]:
aspectos_juntos = reduce(lambda acc, user_table: acc.append(user_table, ignore_index=True), tablas_aspectos)

In [77]:
data_for_model = aspectos_juntos.pivot(index='user', columns='nombre', values='percentil')
data_for_model.head()

nombre,Amabilidad,Amor,Apertura a experiencias,Apertura al cambio,Armonía,Autoexpresión,Autotranscendencia,Conservación,Curiosidad,Desafío,...,Estructura,Extroversión,Familiaridad,Hedonismo,Ideal,Libertad,Practicidad,Rango emocional,Responsabilidad,Superación personal
user,,,,,,,,,,,,,,,,,,,,,
CaroCruzO,0.646199,0.600030,0.476340,0.384629,0.815470,0.315014,0.636343,0.966581,0.876685,0.386312,...,0.910267,0.302479,0.875006,0.562297,0.552993,0.325503,0.695605,0.632731,0.664874,0.817373
Cuadrado,0.996011,0.166496,0.977123,0.076313,0.212282,0.113616,0.721287,0.940948,0.167151,0.419628,...,0.248241,0.593229,0.402426,0.224360,0.209515,0.247657,0.648244,0.196677,0.969324,0.030261
D_Ospina1,0.933331,0.181435,0.896155,0.419867,0.706706,0.060610,0.690733,0.904115,0.382440,0.821216,...,0.473320,0.453246,0.822147,0.187914,0.777369,0.417290,0.674567,0.586806,0.967340,0.394283
DanielSamperO,0.978665,0.322698,0.913175,0.374410,0.812881,0.291185,0.339238,0.838960,0.281041,0.517289,...,0.903781,0.199145,0.734731,0.251318,0.127966,0.711958,0.888425,0.443486,0.710033,0.251885
FALCAO,0.888656,0.296286,0.665561,0.166078,0.691166,0.064613,0.277354,0.847914,0.140321,0.282757,...,0.689561,0.567635,0.628117,0.205201,0.390092,0.244011,0.626617,0.495641,0.948651,0.069001
